# Importación de librerías

In [ ]:
!pip install spacy
!python -m spacy download es_core_news_sm
!pip install transformers
!pip install lightgbm

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string
import joblib

import lightgbm as lgb

from sklearn.model_selection import train_test_split, GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, RocCurveDisplay, ConfusionMatrixDisplay, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier

import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk.collocations import BigramCollocationFinder,BigramAssocMeasures
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('all')

from transformers import pipeline

import spacy
nlp = spacy.load('es_core_news_sm')


# Cargamos el dataset

In [ ]:
df = pd.read_csv('trackings.csv', sep=';')
pd.set_option('display.max_columns', None)
df.describe()

In [ ]:
columnas = df.columns
print(columnas)

# Preparación de datos
## Unificación de comentarios

In [ ]:
df_grouped = df.groupby('IdTracking').apply(
    lambda x: pd.Series({
        'Des': x['Des'].iloc[0],  # Cargar 'Des' solo una vez (el primer valor del grupo)
        'Des_Combined': ' '.join(x['Des.1'].astype(str)),
        'IdTipoTracking': x['IdTipoTracking'].iloc[0],
        'IdPrioridad': x['IdPrioridad'].iloc[0],
        'IdTipoTarea': x['IdTipoTarea'].iloc[0],
        'IdComponente': x['IdComponente'].iloc[0],
        'IdPropuesta': x['IdPropuesta'].iloc[0],
        'IdCuenta': x['IdCuenta'].iloc[0],
        'FecAlt': x['FecAlt'].iloc[0],
        'IdUsuarioAlt': x['IdUsuarioAlt'].iloc[0],
        'FecAsi': x['FecAsi'].iloc[0],
        'IdUsuarioAsi': x['IdUsuarioAsi'].iloc[0],
        'IdEstado': x['IdEstado'].iloc[0],
        'IdTipoEstado': x['IdTipoEstado'].iloc[0],
        'HorEst': x['HorEst'].iloc[0],
        'MinEst': x['MinEst'].iloc[0],
        'Inter': x['Inter'].iloc[0],
        'Exter': x['Exter'].iloc[0],
        'IdHito': x['IdHito'].iloc[0],
        'IdVersion': x['IdVersion'].iloc[0],
        'FecVto': x['FecVto'].iloc[0],
        'HorDes': x['HorDes'].iloc[0],
        'IdProducto': x['IdProducto'].iloc[0],
        'IdCategoria': x['IdCategoria'].iloc[0],
        'AplMan': x['AplMan'].iloc[0],
        'Hordesdes': x['Hordesdes'].iloc[0],
        'Hordestes': x['Hordestes'].iloc[0],
        'Horesttes': x['Horesttes'].iloc[0],
        'Horestdes': x['Horestdes'].iloc[0],
        'HorCli': x['HorCli'].iloc[0],
        'ConCan': x['ConCan'].iloc[0],
        'Obs': x['Obs'].iloc[0],
        'IdTareaProyecto': x['IdTareaProyecto'].iloc[0],
        'IdSubtipoProducto': x['IdSubtipoProducto'].iloc[0],
        'IdTrackingRelacionado': x['IdTrackingRelacionado'].iloc[0],
        'IdComplejidad': x['IdComplejidad'].iloc[0],
        'FecUltMod': x['FecUltMod'].iloc[0],
        'IdServidor': x['IdServidor'].iloc[0],
        'Facturable': x['Facturable'].iloc[0]
    })
).reset_index()

df_grouped.head()


## Preparación del texto

In [ ]:
def limpiar_texto(text):
  # Convertir a minúsculas
  text = text.lower()
  # Eliminar signos de puntuación
  text = text.translate(str.maketrans('', '', string.punctuation))
  # Reemplazar tildes
  text = text.replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('Á', 'A').replace('É', 'E').replace('Í', 'I').replace('Ó', 'O').replace('Ú', 'U')
  # Eliminar números y caracteres especiales
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  # Eliminar espacios en blanco adicionales
  text = ' '.join(text.split())
  # Tokenizar el texto
  tokens = word_tokenize(text, language='spanish')
  # Eliminar stop words
  stop_words = set(stopwords.words('spanish'))
  tokens = [word for word in tokens if not word in stop_words]
  # Lematizar las palabras
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  # Unir los tokens de nuevo en una cadena
  processed_text = ' '.join(tokens)
  return processed_text

df_grouped['Des'] = df_grouped['Des'] + ' ' + df_grouped['Des_Combined']
df_grouped['Des'] = df_grouped['Des'].apply(limpiar_texto)
df_grouped = df_grouped.drop(columns=['Des_Combined'])

df_grouped.head()

### Realizamos un análisis de Sentimientos de la variable Des

In [48]:

# Cargar el modelo de análisis de sentimientos
sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Función para analizar el sentimiento de un texto
def analyze_sentiment(text):
  #Se trunca el texto a un máximo de 512 tokens para que se ajuste al modelo BERT.
  result = sentiment_pipeline(text[:512])
  return result[0]['label'], result[0]['score']

# Aplicar la función al DataFrame
df_grouped[['sentiment_label', 'sentiment_score']] = df_grouped['Des'].apply(lambda x: pd.Series(analyze_sentiment(x)))

# Mapear las etiquetas de sentimiento a valores numéricos
sentiment_mapping = {'1 star': 1, '2 stars': 2, '3 stars': 3, '4 stars': 4, '5 stars': 5}
df_grouped['des_sentimiento'] = df_grouped['sentiment_label'].map(sentiment_mapping)


### Realizamos una Clasificación de texto de la variable Des

In [ ]:
# Cargar el pipeline para clasificación de texto con BERT
classifier = pipeline('text-classification', model='dccuchile/bert-base-spanish-wwm-uncased')

# Función para clasificar el texto
def classify_text(text):
  result = classifier(text[:775])
  return result[0]['label'], result[0]['score']

# Aplicar la función al DataFrame
df_grouped[['classification_label', 'classification_score']] = df_grouped['Des'].apply(lambda x: pd.Series(classify_text(x)))
df_grouped['des_clasificacion'] = df_grouped['classification_label'].map({'LABEL_0': False, 'LABEL_1': True})


## Agregamos la variable FecAlt_diff_minutes que representa el tiempo en minutos desde que un issue fue creado hasta su primera atención

In [50]:
# Convertir las columnas 'FecAlt' y 'FecAlt.1' a formato datetime
df['FecAlt'] = pd.to_datetime(df['FecAlt'], format='%d/%m/%Y %H:%M')
df['FecAlt.1'] = pd.to_datetime(df['FecAlt.1'], format='%d/%m/%Y %H:%M')


In [ ]:
# Find the minimum FecAlt for each IdTracking in df_grouped
min_fecalt = df.groupby('IdTracking')['FecAlt.1'].min().reset_index()
df_diff_min = df_grouped
# Merge the minimum FecAlt values back into the original DataFrame
df_diff_min = df_diff_min.merge(min_fecalt, on=['IdTracking'], how='left')

#Convertimos las columnas a fechas
df_diff_min['FecAlt'] = pd.to_datetime(df_diff_min['FecAlt'], format='%d/%m/%Y %H:%M')
df_diff_min['FecAlt.1'] = pd.to_datetime(df_diff_min['FecAlt.1'], format='%d/%m/%Y %H:%M')

# Calculate the difference in minutes between FecAlt and FecAlt.1
df_grouped['FecAlt_diff_minutes'] = (df_diff_min['FecAlt.1'] - df_diff_min['FecAlt']).dt.total_seconds() / 60

df_grouped.head()

### Eliminamos las columnas no deseadas

In [ ]:


# Eliminar las columnas especificadas
df_grouped = df_grouped.drop(columns=[
      'IdTipoTarea', 'IdComponente', 'IdPropuesta', 'FecAlt','IdUsuarioAlt', 'FecAsi', 'IdUsuarioAsi', 'IdTipoEstado',
      'MinEst', 'IdHito', 'IdVersion', 'FecVto', 'HorDes', 'AplMan', 'Hordesdes',
      'Hordestes', 'Horesttes', 'Horestdes', 'HorCli', 'Obs','IdTareaProyecto', 'IdSubtipoProducto', 'IdTrackingRelacionado',
      'FecUltMod', 'IdServidor', 'sentiment_label','sentiment_score', 'classification_score', 'classification_label'
])

# Mostrar las primeras filas del DataFrame para verificar el resultado
df_grouped.head()


## Corregimos los valores erróneos de la variable Facturable

In [ ]:
# Reemplazamos los #n/a por 0
df_grouped['Facturable'] = df_grouped['Facturable'].replace('#N/D', 0)
df_grouped['Facturable'].value_counts()

## Vectorizamos la variable Des

In [ ]:

# Vectorización con TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=500, ngram_range=(1, 1), tokenizer=word_tokenize)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_grouped['Des'])

# Convertir la matriz TF-IDF a un DataFrame de pandas
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Concatenar el DataFrame de TF-IDF con el DataFrame original
df_grouped = pd.concat([df_grouped, tfidf_df], axis=1)

# Mostrar las primeras filas del DataFrame
df_grouped.shape


## Entrenamos el modelo

In [ ]:
df_grouped['Facturable'] = df_grouped['Facturable'].astype(int)
# Import the necessary library

# Define features and target variable
X = df_grouped.drop(['Facturable', 'Des'], axis=1)
y = df_grouped['Facturable']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Create a LightGBM dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

# Define the parameters for the model
# OrderedDict([('colsample_bytree', 0.9967880717646387), ('learning_rate', 0.0977729735815334), ('max_depth', 10), ('n_estimators', 161), ('num_leaves', 31), ('subsample', 0.7797370559774017)]) <- optimización bayesiana
# OrderedDict([('colsample_bytree', 0.8269448351098192), ('learning_rate', 0.05251549392151682), ('max_depth', 9), ('n_estimators', 100), ('num_leaves', 27), ('subsample', 0.5)]) <- optimización bayesiana
# {'feature_fraction': 1.0, 'learning_rate': 0.05, 'n_estimators': 300, 'num_leaves': 50} <- se obtuvo por GridSearch
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'colsample_bytree': 0.9967880717646387,
    'num_leaves': 31,
    'max_depth': 10,
    'subsample': 0.7797370559774017,
    'learning_rate': 0.0977729735815334,
    'feature_fraction': 1.0,
    'n_estimators': 161
}


# Entrenamos el modelo
model = lgb.train(params,
                  train_data,
                  num_boost_round=100,
                  valid_sets=[test_data],
                  callbacks=[lgb.early_stopping(stopping_rounds=10)]) # Use early_stopping callback

# Predecimos el subconjunto Test
y_pred = model.predict(X_test)

# Convert probabilities to binary predictions
y_pred_binary = [1 if p > 0.50 else 0 for p in y_pred]

# Evaluate the model
print(classification_report(y_test, y_pred_binary))
print(confusion_matrix(y_test, y_pred_binary))


## Dibujamos el area bajo la curva ROC

In [ ]:
# Obtener las tasas de falsos positivos, tasas de verdaderos positivos y umbrales
fpr, tpr, thresholds = roc_curve(y_test, y_pred)

# Calcular el área bajo la curva ROC
roc_auc = auc(fpr, tpr)

# Dibujar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (área = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
plt.show()


## Creamos la matriz de confusión

In [ ]:
# Dibujar la matriz de confusión
cm = confusion_matrix(y_test, y_pred_binary)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['No Facturable', 'Facturable'])
disp.plot()
plt.show()

# Uso del modelo en otros scripts

In [ ]:
# Specify the file path to save the model
model_path = 'model.pkl'

# Save the model
joblib.dump(model, model_path)

In [61]:


# Specify the file path of the saved model
model_path = 'model.pkl'

# Load the model
loaded_model = joblib.load(model_path)

# Now you can use the loaded model for predictions